In [33]:
import pandas as pd
import py7zr
import RAKE

In [34]:
# Extract tsv files
with py7zr.SevenZipFile('data/train.tsv.7z', mode='r') as z:
    z.extract(path="data")
with py7zr.SevenZipFile('data/test.tsv.7z', mode='r') as z:
    z.extract(path="data")

In [35]:
data_df = pd.read_csv("data/train.tsv", sep='\t')

In [36]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

num_attribs = []
cat_attribs = ['name', 'category_name', 'brand_name', 'item_description']

num_pipeline = Pipeline([
    ('label', OrdinalEncoder(handle_unknown='ignore')),
    ('std_scaler', StandardScaler())
])

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),   #returns a dense matrix
    ('cat', OneHotEncoder(), cat_attribs) #returns a sparse matrix
])

In [37]:
data_df = data_df.drop('shipping',axis=1)
data_df = data_df.dropna()
data_df.head()

,train_id,name,item_condition_id,category_name,brand_name,price,item_description
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,Adorable top with a hint of lace and a key hol...
6,6,Acacia pacific tides santorini top,3,Women/Swimwear/Two-Piece,Acacia Swimwear,64.0,Size small but straps slightly shortened to fi...
7,7,Girls cheer and tumbling bundle of 7,3,Sports & Outdoors/Apparel/Girls,Soffe,6.0,You get three pairs of Sophie cheer shorts siz...
8,8,Girls Nike Pro shorts,3,Sports & Outdoors/Apparel/Girls,Nike,19.0,Girls Size small Plus green. Three shorts total.


In [55]:
from rake_nltk import Rake

r = Rake()

def rake_implement(x):
    string = ""
    r.extract_keywords_from_text(x)
    # temp is list of keywords from the above function
    temp = r.get_ranked_phrases()

    # concats the list items to a string
    for i in temp:
        string = string + i + " "
    return string

for column in cat_attribs:
    data_df[column] = data_df[column].apply(lambda x: rake_implement(x))
data_df.head()

C:\Users\Willi\AppData\Local\Temp\ipykernel_29832\2118293445.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[column] = temp_df[column].apply(lambda x: rake_implement(x))


,train_id,name,item_condition_id,category_name,brand_name,price,item_description
1,1,razer blackwidow chroma keyboard,3,tablets parts electronics computers components,razer,52.0,razer synapse app works like work perfectly gr...
2,2,viv blouse ava,1,women tops blouses blouse,target,10.0,pale pink key hole adorable top 3x available w...
6,6,acacia pacific tides santorini top,3,women two swimwear piece,acacia swimwear,64.0,straps slightly shortened size small perfect c...
7,7,tumbling bundle girls cheer 7,3,sports outdoors girls apparel,soffe,6.0,boy shorts spandex matching sets sophie cheer ...
8,8,girls nike pro shorts,3,sports outdoors girls apparel,nike,19.0,girls size small plus green three shorts total


In [56]:
X_train = data_df[:100000].copy()
y_train = X_train['price'].copy()
X_train = X_train.drop('price', axis=1)

X_test = data_df[50000:52000].copy()
y_test = X_test[50000:52000]['price'].copy()
X_test = X_test.drop('price', axis=1)

In [57]:
full_pipeline = full_pipeline.fit(X_train)
X_train_prep = full_pipeline.transform(X_train)
X_test_prep = full_pipeline.transform(X_test)

In [58]:
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(n_neighbors=10)

knn.fit(X_train, y_train)

ValueError: could not convert string to float: 'Razer BlackWidow Chroma Keyboard'

In [ ]:
predictions = knn.predict(X_train_prep)

In [ ]:
X_test['new_price'] = predictions
X_test

ValueError: Length of values (50000) does not match length of index (2000)